In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd

In [ ]:
# 연도 별 영화데이터를 담을 클래스
class Movie_of_Year():
    def __init__(self, inst, year):
        self.inst = inst # 객체 구분용
        self.year = year # 연도
        self.html = None # bs html 
        self.titles = None # 제목
        self.ids = None # Daum에서 정한 영화 id
        self.netizen_score = [] # 관객 점수
        self.critic_score = [] # 평론가 점수
        self.total_audience = [] # 총 관객수
    
    # Html 
    def setHtml(self):
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; InteSl Mac O X 10_10_1) \
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        # 고정된 링크에 연도만 붙여주면 연도별 top 100 영화목록에 접근할 수 있음
        url = "https://movie.daum.net/boxoffice/yearly?year=" + self.year
        response = requests.get(url, headers = headers)
        if response.status_code == 200:
            print('request comfirmed!')
        self.html = bs(response.text,'html.parser')
        return self
    
    # 제목 
    def setTitles(self):
        movie_list = self.html.findAll('a',{'class':'name_movie #title'})
        # movie_list['href']은 주소값들을 주기 때문에 22번째부터 등장하는 id를 가져옴. id가 없는 경우에는 None 처리
        self.ids = [i['href'][22:] if len(i['href'][22:])>1 else 'None' for i in movie_list ]
        self.titles = [i.text for i in movie_list]
        return self
    
    # 점수 
    def setScore(self):
        # info_grade를 find_all하면 연도 별 100개의 점수 쌍을 찾을 수 있음
        # 각 iteration 마다 영화 하나의 점수를 가져오는 것.
        info_grade = self.html.findAll('span',{'class':'info_grade'})
        for info in info_grade: 
            movie_grades=info.findAll('span',{'class':'num_grade'})
            # 점수가 6.8이면 '06', '.' , '08'로 저장되어있어서 처리해줌
            movie_scores = [str(int(i.text)) for i in movie_grades if i.text.isdigit()]
            # 항상 2번째와 4번째 movie_scores가 네티즌, 평론가로 고정되어있음. 평론가 점수는 없을 때 공백으로 처리되어있음
            netizen = '.'.join(movie_scores[:2])
            self.netizen_score.append(netizen)
            critic = '.'.join(movie_scores[2:4])
            if len(critic) == 0:
                critic = 'None'
            self.critic_score.append(critic)
        return self
    
    # 총 관객수 
    def setAudience(self):
        # 영화 id를 link뒤에 붙이면 특이하게도 json data를 받을 수 있음
        # json 라이브러리로 쉽게 뽑아올 수 있음
        for i in self.ids:
            link = 'https://movie.daum.net/moviedb/main/totalAudience.json?movieId='
            total_link = link + i
            try:
                parser = requests.get(total_link).text
                dic = json.loads(parser)
                total_aud = dic['totalAudience']   
            except:
                total_aud = 'None'
            self.total_audience.append(total_aud)
        return self
    
    # 그동안 쌓아온 attributes들을 골라서 불러오기
    def getData(self):
        # 연도, 제목, 평점, 누적 관객수만을 원함
        movie_data = self.__dict__.copy()
        del_li = ['html','ids','inst']
        for i in del_li:
            del movie_data[i]
        return movie_data

In [ ]:
my_objects = []

df = pd.DataFrame()
for i, j in enumerate(['2018','2019']):
    obj = Movie_of_Year(i,j)
    my_objects.append(obj)
    obj.setHtml()
    obj.setTitles()
    obj.setScore()
    obj.setAudience()
    movie_data = obj.getData()
    movie_data = pd.DataFrame(movie_data,index=obj.ids)
    df = pd.concat([df,movie_data])